In [1]:
from tohu import Integer, HashDigest, FakerGenerator
from tohu.custom_generator_NEW import CustomGeneratorNEW
from tohu.utils import print_generated_sequence

## Custom generator without `__init__` method

The simplest way to use `CustomGeneratorNEW` is with only class attributes (without `__init__` method).

In [2]:
class QuuxGenerator(CustomGeneratorNEW):
    aa = Integer(100, 200)
    bb = HashDigest(length=6)
    cc = FakerGenerator(method="name")

In [3]:
g = QuuxGenerator()

2020-04-30 13:18:29 INFO  [DDD] Inside CustomGeneratorNEW.__init__()


In [4]:
print_generated_sequence(g, num=5, seed=11111, sep="\n")

Generated sequence:

Quux(aa=163, bb='7551AA', cc='Michelle Miller')
Quux(aa=171, bb='54596E', cc='Eddie Davis')
Quux(aa=142, bb='2A16D0', cc='Kathleen Lucas')
Quux(aa=140, bb='FDCDD3', cc='Jason Rodriguez')
Quux(aa=121, bb='BDE283', cc='Andrew Pitts')


Note that while the _instance_ `g` contains attributes with the same names `"aa"` and `"bb"` as the parent class `QuuxGenerator`, internally the instance attributes are actually spawned versions of the class attributes (in particular, they are not identical):

In [5]:
print(QuuxGenerator.aa)
print(g.aa)
assert QuuxGenerator.aa is not g.aa

print(QuuxGenerator.bb)
print(g.bb)
assert QuuxGenerator.bb is not g.bb

print(QuuxGenerator.cc)
print(g.cc)
assert QuuxGenerator.cc is not g.cc

<Integer (id=373578)>
<Integer (id=2c03e1)>
<HashDigest (id=6f1fff)>
<HashDigest (id=3362ba)>
<FakerGenerator (id=aa540c)>
<FakerGenerator (id=486a74)>


## Custom generator with `__init__` method

Tohu generators that are defined as instance attributes within the `__init__` method are also picked up.

In [6]:
class QuuxGenerator(CustomGeneratorNEW):
    aa = Integer(100, 200)
    bb = HashDigest(length=6)
    
    def __init__(self, method):
        self.cc = FakerGenerator(method=method)

In [7]:
g = QuuxGenerator(method="first_name")

2020-04-30 13:18:29 INFO  [DDD] Inside CustomGeneratorNEW.__init__()


In [8]:
print_generated_sequence(g, num=5, seed=11111, sep="\n")

Generated sequence:

Quux(aa=163, bb='7551AA', cc='Mary')
Quux(aa=171, bb='54596E', cc='Jennifer')
Quux(aa=142, bb='2A16D0', cc='Christopher')
Quux(aa=140, bb='FDCDD3', cc='Joseph')
Quux(aa=121, bb='BDE283', cc='Kelly')


## Custom generators and loop variables

Custom generator provides a classmethod to register loop variables with the class. Once an instance of the custom generator class is instantiated, the method `extract_loop_runner()` returns a loop runner containing spawns of the previously registered loop variables. This is used internally to implement looped custom generators.

In [9]:
from tohu.looping_NEW import LoopVariableNEW

In [10]:
xx = LoopVariableNEW("xx", values=[111, 222, 333]).set_loop_level(42)

In [11]:
class QuuxGenerator(CustomGeneratorNEW):
    aa = Integer(100, 200)

QuuxGenerator.register_loop_variable(xx)

In [12]:
QuuxGenerator._tohu_cg_class_loop_variables

[<LoopVariable: name='xx', loop_level=42, values=[111, 222, 333], cur_value=111 (tohu_id=777cdd)>]

In [13]:
g = QuuxGenerator()

2020-04-30 13:18:29 INFO  [DDD] Inside CustomGeneratorNEW.__init__()


In [14]:
loop_runner = g.extract_loop_runner()

In [15]:
loop_runner.loop_variables

{'xx': <LoopVariable: name='xx', loop_level=42, values=[111, 222, 333], cur_value=111 (tohu_id=5667af)>}

---

## Looped custom generator

In [16]:
from tohu.foreach_NEW import foreach_NEW

In [17]:
@foreach_NEW(xx=["AAA", "BBB"])
@foreach_NEW(yy=[111, 222, 333])
class QuuxGenerator(CustomGeneratorNEW):
    aa = xx
    bb = yy
    cc = Integer(100, 200)
    dd = HashDigest(length=6)

Internally, the `QuuxGenerator` class has been augmented by the `@foreach_NEW` decorators with information about the loop level (here: 2, since we have two nested decorators) and the loop variables present.

In [18]:
QuuxGenerator

<Looped custom generator class: 'QuuxGenerator', wrapped using @foreach>

In [19]:
QuuxGenerator.loop_level

2

Loop variables:

In [20]:
QuuxGenerator.loop_variables

{'yy': <LoopVariable: name='yy', loop_level=1, values=[111, 222, 333], cur_value=111 (tohu_id=394357)>,
 'xx': <LoopVariable: name='xx', loop_level=2, values=['AAA', 'BBB'], cur_value='AAA' (tohu_id=909d2c)>}

In [21]:
g = QuuxGenerator()

2020-04-30 13:18:29 INFO  [DDD] Inside CustomGeneratorNEW.__init__()


In [22]:
g.custom_gen_instance._loop_runner.loop_variables

{'yy': <LoopVariable: name='yy', loop_level=1, values=[111, 222, 333], cur_value=111 (tohu_id=df72b3)>,
 'xx': <LoopVariable: name='xx', loop_level=2, values=['AAA', 'BBB'], cur_value='AAA' (tohu_id=0fad0e)>}

In [23]:
g.custom_gen_instance._loop_runner.loop_variables["xx"].rewind_loop_variable()
print_generated_sequence(g.custom_gen_instance, num=5, seed=11111, sep="\n")
g.custom_gen_instance._loop_runner.loop_variables["xx"].advance()
print_generated_sequence(g.custom_gen_instance, num=5, seed=11111, sep="\n")

Generated sequence:

Quux(aa='AAA', bb=111, cc=163, dd='7551AA')
Quux(aa='AAA', bb=111, cc=171, dd='54596E')
Quux(aa='AAA', bb=111, cc=142, dd='2A16D0')
Quux(aa='AAA', bb=111, cc=140, dd='FDCDD3')
Quux(aa='AAA', bb=111, cc=121, dd='BDE283')
Generated sequence:

Quux(aa='BBB', bb=111, cc=163, dd='7551AA')
Quux(aa='BBB', bb=111, cc=171, dd='54596E')
Quux(aa='BBB', bb=111, cc=142, dd='2A16D0')
Quux(aa='BBB', bb=111, cc=140, dd='FDCDD3')
Quux(aa='BBB', bb=111, cc=121, dd='BDE283')


In [24]:
#g._loop_runner.loop_variables["xx"].advance()

Both the parent class `QuuxGenerator` and its instance `g` contain loop runners containing loop variables `xx` and `yy`.

In [25]:
QuuxGenerator.custom_gen_cls._tohu_cg_class_loop_variables

[<LoopVariable: name='yy', loop_level=1, values=[111, 222, 333], cur_value=111 (tohu_id=394357)>,
 <LoopVariable: name='xx', loop_level=2, values=['AAA', 'BBB'], cur_value='AAA' (tohu_id=909d2c)>]

In [26]:
g.custom_gen_instance._loop_runner.loop_variables

{'yy': <LoopVariable: name='yy', loop_level=1, values=[111, 222, 333], cur_value=111 (tohu_id=df72b3)>,
 'xx': <LoopVariable: name='xx', loop_level=2, values=['AAA', 'BBB'], cur_value='BBB' (tohu_id=0fad0e)>}

In [27]:
assert g.custom_gen_instance._loop_runner.loop_variables["xx"].is_clone_of(QuuxGenerator.custom_gen_cls._tohu_cg_class_loop_variables[1])
assert g.custom_gen_instance._loop_runner.loop_variables["yy"].is_clone_of(QuuxGenerator.custom_gen_cls._tohu_cg_class_loop_variables[0])